## Test

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp	

In [ ]:
# Create a new model
m = gp.Model("matrix1") 

In [ ]:
# Create variables
x = m.addMVar(shape=3, vtype=GRB.BINARY, name="x")

In [7]:
# Build (sparse) constraint matrix
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

In [8]:
print(A)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (2, 3)>
  Coords	Values
  (0, 0)	1.0
  (0, 1)	2.0
  (0, 2)	3.0
  (1, 0)	-1.0
  (1, 1)	-1.0


In [9]:
# Build rhs vector
rhs = np.array([4.0, -1.0])

In [10]:
# Add constraints
m.addConstr(A @ x <= rhs, name="c")

<MConstr (2,) *awaiting model update*>

In [11]:
# Set objective
obj = np.array([1.0, 1.0, 2.0])
m.setObjective(obj @ x, GRB.MAXIMIZE)

In [12]:
# Optimize model
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Linux Mint 21.3")

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x8d4960d3
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [13]:
print(x.X)

[1. 0. 1.]


In [14]:
print('Obj: %g' % m.ObjVal)

Obj: 3


## Project Pfizer Lab 1

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# --- 1. Define Sets and Parameters ---

# Sets
bricks = list(range(1, 23)) # Bricks 1 to 22 [cite: 136]
srs = list(range(1, 5))      # SRs 1 to 4 [cite: 136]

# Parameters
# Workload (Index Values) from Table 2 [cite: 143]
workload = {
    1: 0.1609, 2: 0.1164, 3: 0.1026, 4: 0.1516, 5: 0.0939, 6: 0.1320, 7: 0.0687,
    8: 0.0930, 9: 0.2116, 10: 0.2529, 11: 0.0868, 12: 0.0828, 13: 0.0975, 14: 0.8177,
    15: 0.4115, 16: 0.3795, 17: 0.0710, 18: 0.0427, 19: 0.1043, 20: 0.0997,
    21: 0.1698, 22: 0.2531
}

# Distances from Table 3 [cite: 187]
distances = {
    (1, 1): 16.16, (1, 2): 24.08, (1, 3): 24.32, (1, 4): 21.12,
    (2, 1): 19.00, (2, 2): 26.47, (2, 3): 27.24, (2, 4): 17.33,
    (3, 1): 25.29, (3, 2): 32.49, (3, 3): 33.42, (3, 4): 12.25,
    (4, 1): 0.00,  (4, 2): 7.93,  (4, 3): 8.31,  (4, 4): 36.12,
    (5, 1): 3.07,  (5, 2): 6.44,  (5, 3): 7.56,  (5, 4): 37.37,
    (6, 1): 1.22,  (6, 2): 7.51,  (6, 3): 8.19,  (6, 4): 36.29,
    (7, 1): 2.80,  (7, 2): 10.31, (7, 3): 10.95, (7, 4): 33.50,
    (8, 1): 2.87,  (8, 2): 5.07,  (8, 3): 5.67,  (8, 4): 38.80,
    (9, 1): 3.80,  (9, 2): 8.01,  (9, 3): 7.41,  (9, 4): 38.16,
    (10, 1): 12.35, (10, 2): 4.52,  (10, 3): 4.35,  (10, 4): 48.27,
    (11, 1): 11.11, (11, 2): 3.48,  (11, 3): 2.97,  (11, 4): 47.14,
    (12, 1): 21.99, (12, 2): 22.02, (12, 3): 24.07, (12, 4): 39.86,
    (13, 1): 8.82,  (13, 2): 3.30,  (13, 3): 5.36,  (13, 4): 43.31,
    (14, 1): 7.93,  (14, 2): 0.00,  (14, 3): 2.07,  (14, 4): 43.75,
    (15, 1): 9.34,  (15, 2): 2.25,  (15, 3): 1.11,  (15, 4): 45.43,
    (16, 1): 8.31,  (16, 2): 2.07,  (16, 3): 0.00,  (16, 4): 44.43,
    (17, 1): 7.31,  (17, 2): 2.44,  (17, 3): 1.11,  (17, 4): 43.43,
    (18, 1): 7.55,  (18, 2): 0.75,  (18, 3): 1.53,  (18, 4): 43.52,
    (19, 1): 11.13, (19, 2): 18.41, (19, 3): 19.26, (19, 4): 25.40,
    (20, 1): 17.49, (20, 2): 23.44, (20, 3): 24.76, (20, 4): 23.21,
    (21, 1): 11.03, (21, 2): 18.93, (21, 3): 19.28, (21, 4): 25.43,
    (22, 1): 36.12, (22, 2): 43.75, (22, 3): 44.43, (22, 4): 0.00
}

# Current Assignments from Table 1 [cite: 141]
current_assignment_map = {
    1: [4, 5, 6, 7, 8, 15],
    2: [10, 11, 12, 13, 14],
    3: [9, 16, 17, 18],
    4: [1, 2, 3, 19, 20, 21, 22]
}

In [ ]:
# --- 2. Create Model and Variables ---

# Initialize the model
m = gp.Model("SR_Territories")

# x[i, j] = 1 if brick i is assigned to SR j
x = m.addVars(bricks, srs, vtype=GRB.BINARY, name="x")

print(len(bricks))
print(len(srs))

22
4


In [ ]:
# --- 3. Add Constraints ---

# Constraint 1: Each brick (i) is assigned to exactly one SR (j)
m.addConstrs(
    (x.sum(i, '*') == 1 for i in bricks), name="AssignBrick"
)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

In [ ]:
# Constraint 2: Workload Balancing
wl_min = 0.8
wl_max = 1.2

# Add the lower bound constraints (>= wl_min) for all j in srs
m.addConstrs(
    (gp.quicksum(workload[i] * x[i, j] for i in bricks) >= wl_min for j in srs), 
    name="WorkloadMin"
)

# Add the upper bound constraints (<= wl_max) for all j in srs
m.addConstrs(
    (gp.quicksum(workload[i] * x[i, j] for i in bricks) <= wl_max for j in srs), 
    name="WorkloadMax"
)

print("Added brick assignment and workload constraints.")

Added brick assignment and workload constraints.


In [ ]:
obj_distance = 0 # Start at zero
for i in bricks:
    for j in srs:
        obj_distance += distances[i, j] * x[i, j] # Use += to add